In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!pip install python_dotenv

## Packages

In [42]:
import torch
import pandas as pd
import re
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
import os
import json

### Load environment variables

In [29]:
base_path = "drive/MyDrive/Colab Notebooks"

load_dotenv(dotenv_path=f"{base_path}/hugging_face_token/.env")

True

### Transformation functions

In [91]:
def read_categories_examples(path: str = f"{base_path}/dataset/output/songs.csv") -> str:
    songs = pd.read_csv(path, sep=";")
    categories = songs['category'].unique().tolist()
    mode = get_avg_wordcount_by_genre()
    examples = ""
    for idx, cat in enumerate(categories):
        example = songs[songs['category'] == cat].sample(1).iloc[0]['transformed']
        examples += f"Song {idx} \n -category: {cat} \n -lyric: {example[:300]} \n -mode: {int(mode.get(cat))} \n\n"
    return examples

def get_examples_to_classify(path: str = f"{base_path}/dataset/output/songs.csv") -> tuple[dict, dict]:
    songs = pd.read_csv(path, sep=";")
    categories = songs['category'].unique().tolist()
    test_examples = {}
    labels = {}
    for idx, cat in enumerate(categories):
        row = songs[songs['category'] == cat].sample(1).iloc[0]
        lyric = row['transformed'][:300]
        test_examples[f"Song {idx}"] = lyric
        labels[f"Song {idx}"] = cat
    return test_examples, labels

def parse_llm_response(response_text: str) -> dict:
    predictions = {}
    for line in response_text.strip().split("\n"):
        match = re.match(r"Song (\d+):\s*(.+)", line.strip())
        if match:
            song_id = f"Song {match.group(1)}"
            predicted_genre = match.group(2).strip()
            predictions[song_id] = predicted_genre
    return predictions

def get_avg_wordcount_by_genre(path: str = f"{base_path}/dataset/output/songs.csv") -> dict:
    songs = pd.read_csv(path, sep=";")
    avg_len = songs.groupby('category')['len'].agg(lambda x: x.mode().iloc[0])
    return dict(avg_len)

### LLM model client

In [26]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [27]:
client = InferenceClient(model_id, provider="together")

### Taking music examples by category

In [89]:
train_examples = read_categories_examples()

In [123]:
prompt_sistema = (
"Você é um especialista em música brasileira. \n"
"Sua tarefa é identificar o gênero musical de letras de músicas. \n"
f"Use como referência os exemplos abaixo, considerando a letra da músca, a categoria da música e o a moda da quantidade de palavras distintas por gênero. \n"
"Sempre retorne **apenas UM gênero** por letra.\n\n"
f"{train_examples}"
)

In [122]:
print(prompt_sistema)

Você é um especialista em música brasileira. 
Sua tarefa é identificar o gênero musical de letras de músicas. 
Use como referência os exemplos abaixo, considerando a letra da músca, a categoria da música e o a moda da quantidade de palavras distintas por gênero. 
Sempre retorne **apenas UM gênero** por letra.

Song 0 
 -category: Axé 
 -lyric: corpo ficou lembranca esquecer gente transa andado procura hora sonhar importa loucura encontrar saudade beijo gostoso daqueles momentos sentimentos delirios amor coisas vida pede voltar apagar manda gente coracao manda corpo tentacao terra mar levar mundo juntar queria noite mostrar amo mede desejo 
 -mode: 187 

Song 1 
 -category: Black Music 
 -lyric: pregue palavra deus acreditas prepare futuro faca formigas verao estocam comida inverno sentarem rainha eternidade covarde peito subir majestade queres viver trezentos anos viva va muita fe acreditas rosto manchado esforco poeira suor sangue touro forte robusto coracao justo alma arrependida lev

### Taking music examples to test

In [99]:
test_examples, ground_truth = get_examples_to_classify()

In [138]:
prompt_usuario = f"""
Você deve escolher **apenas UM** gênero musical que melhor representa **cada** letra.\n
Você tem essas opções: Axé, Black Music, Bossa Nova, Folk, Forró, Funk,
    Religioso, Infantil, Jovem Guarda, MPB, Pagode, Pop,
    Rap, Reggae \n

Responda exatamente no seguinte formato (sem explicações ou variações): \n
Song X: gênero \n\n

Exemplo: \n
Song 0: Samba \n
Song 1: MPB \n
... \n

Agora classifique as seguintes letras:\n
{test_examples}
        """

In [139]:
print(prompt_usuario)


Você deve escolher **apenas UM** gênero musical que melhor representa **cada** letra.
 
Você tem essas opções: Axé, Black Music, Bossa Nova, Folk, Forró, Funk,
    Religioso, Infantil, Jovem Guarda, MPB, Pagode, Pop,
    Rap, Reggae 


Responda exatamente no seguinte formato (sem explicações ou variações): 

Song X: gênero 



Exemplo: 

Song 0: Samba 

Song 1: MPB 

... 


Agora classifique as seguintes letras:

{'Song 0': 'ar ar sinto forte emocao fico paz coracao sofri busca amor perfeito iludi vivendo aprendi amar lindo importa sofrer feliz destino ar amar sofrer sofrer feliz contigo ar amar acreditar sonho bonito viver crer cantar amar', 'Song 1': 'sinto existe sente viver sinto existe sente viver esperava encontrei procurava gamei ti rainha rei ei mulher dure amor dure bue sinto especial amor existe gente diferente atraente ingrediente vida contente vamos curtir presente viagens frequentes cacando estrela cadente deixa falar ouvido amo deixa ', 'Song 2': 'vestida noiva veu grina

### Building the request

In [140]:
messages = [
        {
            "role": "system",
            "content": prompt_sistema
        },
        {
            "role": "user",
            "content": prompt_usuario
        }
]

In [ ]:
response = client.chat.completions.create(
        messages=messages,
        temperature=1
)

### Evaluating responses

In [144]:
llm_output = response.choices[0].message.content

predictions = parse_llm_response(llm_output)

print("\nComparação LLM x Categoria Real: \n")
correct = 0
total = len(ground_truth)

for song_id, true_label in ground_truth.items():
    pred_label = predictions.get(song_id, "N/A")
    status = "✅" if pred_label.lower() == true_label.lower() else "❌"
    print(f"{song_id}: Predito = {pred_label}, Real = {true_label} {status}")
    if status == "✅":
        correct += 1

print(f"\nAcurácia: {correct}/{total} = {correct / total:.2%}")


Comparação LLM x Categoria Real: 

Song 0: Predito = Bossa Nova, Real = Axé ❌
Song 1: Predito = MPB, Real = Black Music ❌
Song 2: Predito = Bossa Nova, Real = Bossa Nova ✅
Song 3: Predito = Folk, Real = Folk ✅
Song 4: Predito = Forró, Real = Forró ✅
Song 5: Predito = Funk, Real = Funk ✅
Song 6: Predito = Religioso, Real = Religioso ✅
Song 7: Predito = Infantil, Real = Infantil ✅
Song 8: Predito = MPB, Real = Jovem Guarda ❌
Song 9: Predito = MPB, Real = MPB ✅
Song 10: Predito = Pagode, Real = Pagode ✅
Song 11: Predito = Pop, Real = Pop ✅
Song 12: Predito = Folk, Real = Rap ❌
Song 13: Predito = Rap, Real = Reggae ❌

Acurácia: 9/14 = 64.29%
